In [1]:
import sys
sys.path.append('/Users/samrelins/Documents/LIDA/ace_project/')
from src.data_prep import *
from src.train_test import *

# load in ace dataset with text
extra_filepath = '/Users/samrelins/Documents/LIDA/ace_project/data' \
              '/ace_data_extra.csv'
ace_dat_extra = pd.read_csv(extra_filepath)
ace_dat_extra = (ace_dat_extra
                 .pipe(start_pipeline)
                 .pipe(clean_data)
                 .pipe(fill_nas)
                 .pipe(add_features))

orig_filepath = '/Users/samrelins/Documents/LIDA/ace_project/data' \
              '/ace_data_orig.csv'
ace_dat_orig = pd.read_csv(orig_filepath)
ace_dat_orig = (ace_dat_orig
                 .pipe(start_pipeline)
                 .pipe(clean_data)
                 .pipe(fill_nas)
                 .pipe(add_features))

In [2]:
len(ace_dat_orig)

486

In [3]:
len(ace_dat_extra)

488

In [7]:
orig_columns = ace_dat_orig.drop("referral_from", axis=1).columns

def find_similar_rows(extra_idx):
    similar_rows = []
    different_entries = []
    for orig_idx in range(len(ace_dat_orig)):
        extra_row = ace_dat_extra[orig_columns].iloc[extra_idx,:]
        orig_row = ace_dat_orig[orig_columns].iloc[orig_idx,:]
        same_entries = extra_row == orig_row
        if sum(same_entries) > len(orig_columns) - 2:
            row_name = ace_dat_orig.iloc[orig_idx,:].name
            similar_rows.append(row_name)
            different_entries.append(orig_columns[~same_entries])
    return similar_rows, different_entries

def find_similar_rows_new(row_iloc, df_from, df_to_check):
    common_columns = pd.Index([col for col in df_from.columns
                      if col in df_to_check.columns])

    similar_row_locs = []
    different_entries = []

    row_from = df_from[common_columns].iloc[row_iloc,:]
    for loop_idx in range(len(df_to_check)):
        row_to_check = df_to_check[common_columns].iloc[loop_idx,:]
        same_entries = row_from == row_to_check
        if sum(same_entries) > len(common_columns) - 2:
            similar_row_loc = row_to_check.name
            similar_row_locs.append(similar_row_loc)
            different_entries.append(common_columns[~same_entries])
    return similar_row_locs, different_entries

for i in range(len(ace_dat_extra)):
    row_i = ace_dat_extra[orig_columns].iloc[i,:]
    i_name = row_i.name
    same_rows = (ace_dat_orig[orig_columns] == row_i).all(axis=1)
    if not sum(same_rows):
        similar_rows, different_entries = find_similar_rows(i)
        if len(similar_rows):
            if different_entries[0] in ["temp", "heart_rate", "ox_sat",
                                        "resp_rate"]:
                num_feature = different_entries[0]
                extra_fig = ace_dat_extra.iloc[i,:][num_feature].values[0]
                orig_fig = ace_dat_orig.loc[similar_rows,
                           :][num_feature].values[0]
                if abs(extra_fig - orig_fig) > 0.01:
                    print(f"{i_name} -> {similar_rows} : {num_feature}"
                          + f" {extra_fig} -> {orig_fig}")
            else:
                print(f"{i_name} -> {similar_rows} : {different_entries}")
        else:
            print(f"{i_name} has no similar examples")
    if sum(same_rows) > 1:
        other_rows_iloc = same_rows[same_rows].index
        other_rows_name = []
        for iloc in other_rows_iloc:
            other_rows_name.append(ace_dat_orig.iloc[iloc,:].name)
        print(f"{i_name} = {other_rows_name}")

46 has no similar examples
252 -> [250] : Index(['heart_rate'], dtype='object') 111.0 -> [110.]
426 has no similar examples
170 -> [169] : Index(['heart_rate'], dtype='object') 119.28971962616822 -> [119.36533958]
234 has no similar examples
470 has no similar examples


In [38]:
def check_for_differences(df_from, df_to_check):
    common_columns = pd.Index([col for col in df_from.columns
                      if col in df_to_check.columns])
    for loop_iloc in range(len(df_from)):
        row_i = df_from[common_columns].iloc[loop_iloc,:]
        row_loc = row_i.name
        same_rows = (df_to_check[common_columns] == row_i).all(axis=1)
        if not sum(same_rows):
            similar_rows, different_entries = find_similar_rows_new(i,
                                                                    df_from,
                                                                    df_to_check)
            if len(similar_rows):
                similar_row_is_numeric = np.issubdtype(
                    df_from[different_entries[0][0]].dtype,
                    np.number
                )
                if similar_row_is_numeric:
                    num_feature = different_entries[0]
                    df_from_fig = int(row_i[num_feature])
                    df_to_check_fig = int(df_to_check
                                       .loc[similar_rows,:][num_feature]
                                       .values[0][0])
                    print(df_from_fig - df_to_check_fig)
                    if abs(df_from_fig - df_to_check_fig) > 0.01:
                        print(f"{row_loc} -> {similar_rows} : {num_feature}"
                              + f" {df_from_fig} -> {df_to_check_fig}")
                else:
                    print(f"{row_loc} -> {similar_rows} : {different_entries}")
            else:
                print(f"{row_loc} has no similar examples")
        if sum(same_rows) > 1:
            other_rows_iloc = same_rows[same_rows].index
            other_rows_name = []
            for iloc in other_rows_iloc:
                other_rows_name.append(df_to_check.iloc[iloc,:].name)
            print(f"{i_name} = {other_rows_name}")

check_for_differences(ace_dat_extra, ace_dat_orig)

0
0
0
0
0
0
0
1
23 -> [192] : Index(['temp'], dtype='object') 37 -> 36
1
30 -> [192] : Index(['temp'], dtype='object') 37 -> 36
0
0
1
52 -> [192] : Index(['temp'], dtype='object') 37 -> 36
1
53 -> [192] : Index(['temp'], dtype='object') 37 -> 36
0
0
0
1
80 -> [192] : Index(['temp'], dtype='object') 37 -> 36
0
0
0
1
86 -> [192] : Index(['temp'], dtype='object') 37 -> 36
0
1
90 -> [192] : Index(['temp'], dtype='object') 37 -> 36
1
91 -> [192] : Index(['temp'], dtype='object') 37 -> 36
1
94 -> [192] : Index(['temp'], dtype='object') 37 -> 36
0
2
98 -> [192] : Index(['temp'], dtype='object') 38 -> 36
1
99 -> [192] : Index(['temp'], dtype='object') 37 -> 36
1
101 -> [192] : Index(['temp'], dtype='object') 37 -> 36
1
108 -> [192] : Index(['temp'], dtype='object') 37 -> 36
1
110 -> [192] : Index(['temp'], dtype='object') 37 -> 36
0
0
2
115 -> [192] : Index(['temp'], dtype='object') 38 -> 36
0
0
0
1
130 -> [192] : Index(['temp'], dtype='object') 37 -> 36
0
1
146 -> [192] : Index(['temp'], dtyp

KeyboardInterrupt: 

In [23]:
ace_dat_orig["heart_rate"].isnum

AttributeError: 'Series' object has no attribute 'isnum'

In [27]:
np.issubdtype(ace_dat_orig["heart_rate"].dtype, np.number)

True

In [1]:
from sklearn.linear_model import LogisticRegression
import sys
sys.path.append('/Users/samrelins/Documents/LIDA/ace_project/')
from src.train_test import *

data_loc = "/Users/samrelins/Documents/LIDA/ace_project/data/"
ace_data= pd.read_csv(data_loc + "ace_data_extra.csv")
text_features = ["medical_history", "examination_summary", "recommendation"]
X_train, y_train, X_test, y_test = return_train_test(
    ace_data.drop(text_features, axis=1)
)

Required features missing to run <function add_free_text_features at 0x7f9f14c83f70>


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/samrelins/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/samrelins/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
best_scores, best_params = param_search_classifier(
    X_train=X_train, y_train=y_train,
    clf=LogisticRegression(random_state=0,
                           max_iter=1000,
                           solver="saga",
                           class_weight="balanced",
                           n_jobs=-1),
    param_grid={"penalty": ["none", "l1", "l2"],
                'C': np.logspace(-1, 0.3, 10)},
    scaled=True,
    n_random_iterations=10,
    resample="undersample"
)

Testing LogisticRegression(class_weight='balanced', max_iter=1000, n_jobs=-1,
                   random_state=0, solver='saga') classifier with one_hot encoded features.
{'f1': [0.2597402597402597, 0.18461538461538457, 0.28169014084507044, 0.23529411764705885, 0.38095238095238093, 0.2758620689655173, 0.26315789473684215, 0.23728813559322035, 0.2564102564102564, 0.32, 0.2295081967213115, 0.2580645161290322, 0.24137931034482757, 0.3055555555555555, 0.21333333333333335, 0.2702702702702703, 0.19444444444444445, 0.23076923076923075, 0.22535211267605632, 0.2028985507246377, 0.21052631578947367, 0.2191780821917808, 0.28947368421052627, 0.2535211267605634, 0.22580645161290322, 0.16091954022988506, 0.19999999999999998, 0.23076923076923075, 0.2571428571428571, 0.24324324324324326], 'roc_auc': [0.5085470085470085, 0.44139194139194143, 0.5415140415140416, 0.5183150183150182, 0.6697191697191697, 0.5463980463980463, 0.514041514041514, 0.5076312576312576, 0.503052503052503, 0.586080586080586, 0.49664

 10%|█         | 3/30 [00:08<01:17,  2.85s/it]


KeyboardInterrupt: 

In [8]:
best_scores

{'mean_f1': 0.2639188049306702,
 'std_f1': 0.0648341137778323,
 'mean_roc_auc': 0.5388787138787138,
 'std_roc_auc': 0.0571059327183697,
 'mean_accuracy': 0.6256880733944955,
 'std_accuracy': 0.05881108443422933,
 'mean_recall': 0.4092592592592593,
 'std_recall': 0.11870644822620258,
 'mean_precision': 0.19759574896240356,
 'std_precision': 0.04913439147512597,
 'mean_true_pos': 7.366666666666666,
 'std_true_pos': 2.1367160680716464,
 'mean_true_neg': 60.833333333333336,
 'std_true_neg': 6.904507384471555,
 'mean_false_pos': 30.166666666666668,
 'std_false_pos': 6.904507384471555,
 'mean_false_neg': 10.633333333333333,
 'std_false_neg': 2.1367160680716464}

In [10]:
best_scores

{'mean_f1': 0.24742650334085026,
 'std_f1': 0.05274150381565719,
 'mean_roc_auc': 0.5249999999999999,
 'std_roc_auc': 0.043742865774235966,
 'mean_accuracy': 0.6198776758409785,
 'std_accuracy': 0.043335610125809935,
 'mean_recall': 0.3833333333333334,
 'std_recall': 0.10178448950860218,
 'mean_precision': 0.18443824036791215,
 'std_precision': 0.03666545932510187,
 'mean_true_pos': 6.9,
 'std_true_pos': 1.8321208111548393,
 'mean_true_neg': 60.666666666666664,
 'std_true_neg': 5.399588461684423,
 'mean_false_pos': 30.333333333333332,
 'std_false_pos': 5.399588461684423,
 'mean_false_neg': 11.1,
 'std_false_neg': 1.8321208111548393}

In [12]:
best_scores

{'mean_f1': 0.26813302080251566,
 'std_f1': 0.0456950807594457,
 'mean_roc_auc': 0.5252136752136752,
 'std_roc_auc': 0.05720576408597842,
 'mean_accuracy': 0.5272171253822631,
 'std_accuracy': 0.05854013027474763,
 'mean_recall': 0.5222222222222223,
 'std_recall': 0.09686442096757052,
 'mean_precision': 0.18117037567748512,
 'std_precision': 0.03224250162454631,
 'mean_true_pos': 9.4,
 'std_true_pos': 1.7435595774162693,
 'mean_true_neg': 48.06666666666667,
 'std_true_neg': 6.276587891167904,
 'mean_false_pos': 42.93333333333333,
 'std_false_pos': 6.276587891167904,
 'mean_false_neg': 8.6,
 'std_false_neg': 1.7435595774162693}

In [20]:
extra_data_loc = "/Users/samrelins/Documents/LIDA/ace_project/data" \
               "/ace_data_extra" \
             ".csv"
ace_data_extra = (pd.read_csv(extra_data_loc)
                  .drop(["medical_history", "examination_summary",
                         "recommendation"], axis=1))

X_train_extra, y_train_extra, X_test_extra, y_test_extra = return_train_test\
    (ace_data_extra)

best_extra_scores, best_extra_params = param_search_classifier(
    X_train=X_train_extra, y_train=y_train_extra,
    clf=LogisticRegression(random_state=0,
                           max_iter=1000,
                           solver="saga",
                           class_weight="balanced",
                           n_jobs=-1),
    param_grid={"penalty": ["l1"],
                'C': np.linspace(0.01, 1, 50)},
    scaled=True,
)

best_extra_scores

Testing LogisticRegression(class_weight='balanced', max_iter=1000, n_jobs=-1,
                   random_state=0, solver='saga') classifier with one_hot encoded features.


100%|██████████| 50/50 [00:10<00:00,  4.58it/s]


{'f1': 0.24360587002096437,
 'roc_auc': 0.5407000407000407,
 'accuracy': 0.6299694189602446,
 'recall': 0.40740740740740744,
 'precision': 0.17777777777777778,
 'true_pos': 7.333333333333333,
 'true_neg': 61.333333333333336,
 'false_pos': 29.666666666666668,
 'false_neg': 10.666666666666666}

In [18]:
best_params

{'C': 0.19144819761699575, 'penalty': 'l1'}

In [21]:
best_extra_params

{'C': 0.8787755102040816, 'penalty': 'l1'}

In [40]:
X_train_prepped, X_test_prepped = encode_and_scale(
    X_train=X_train_extra,
    y_train=y_train_extra,
    X_test=X_test_extra,
    cat_encoder="one_hot",
    scaled=True
)

log_reg_clf = LogisticRegression(
    random_state=0,
    max_iter=1000,
    solver="saga",
    class_weight="balanced",
    n_jobs=-1,
    C=0.2,
    penalty="l1"
)

log_reg_clf.fit(X_train_prepped, y_train_extra)

score_classifier(log_reg_clf, X_train_prepped, y_train_extra)

{'f1': 0.3804347826086957,
 'roc_auc': 0.6500814000814001,
 'accuracy': 0.6513761467889908,
 'recall': 0.6481481481481481,
 'precision': 0.2692307692307692,
 'true_pos': 35,
 'true_neg': 178,
 'false_pos': 95,
 'false_neg': 19}

In [44]:
nonzero_coefs = log_reg_clf.coef_[0] > 0
pd.DataFrame({
    "feature": X_train_prepped.columns[nonzero_coefs],
    "value": log_reg_clf.coef_[0][nonzero_coefs]
}).sort_values("value", ascending=False)

,feature,value
6,apls_resp_rate_cat_high,0.334990
0,referral_profession_doctor,0.324761
2,referral_date_winter,0.154355
1,address_bd03,0.148631
4,activity_level_lower,0.069948
3,illness_severity_moderate,0.067535
5,food_allergy_Y,0.034319
7,apls_heart_rate_cat_high,0.010547
